Note - data is coming from this kaggle depot:
https://www.kaggle.com/harlfoxem/housesalesprediction

In [ ]:
%%javascript
// Ensure our output is sufficiently large
IPython.OutputArea.auto_scroll_threshold = 9999;

# Sprint 1

* Load libraries
* Load data
* Explore data for quality
* Data clean-up
* Explore data for Linear Regression Suitability
* Mechanical Feature Engineering
* Run Linear Regression
* Explore coefficients


## Load Libraries

In [ ]:
# Emnsure all charts plotted automatically
%matplotlib inline

import warnings
import numpy as np
import pandas as pd

import datetime
import time

import statsmodels.api as sm

import sklearn

from sklearn import preprocessing

from sklearn_pandas import DataFrameMapper

from xgboost import XGBRegressor

from IPython.display import display, Markdown

import matplotlib.pyplot as plt
import seaborn as sns

# Don't truncate dataframe displays columwise
pd.set_option('display.max_columns', None)

## Engineer Data: Load the data

You load the CSV data into a Pandas object. This is a common Python library to work with data in row/column format, like .csv files.

Print out top 10 rows just to get a feel for the data

In [ ]:
df = None

__You should see:__

* 21613 rows of data across 21 columns (different variables)
* All data appears to be numerical

__Helpful  decoder ring for what each variable means (stuff you would get from the client)__

* __id__ a notation for a house
* __date__ Date house was sold
* __price__ Price is prediction target
* __bedrooms__ Number of Bedrooms/House
* __bathrooms__ Number of bathrooms/House
* __sqft_living__ square footage of the home
* __sqft_lot__ square footage of the lot
* __floors__ Total floors (levels) in house
* __waterfront__ House which has a view to a waterfront
* __view__ quality of view - higher == better
* __condition__ How good the condition is ( Overall ) - higher == better
* __grade__ overall grade given to the housing unit, based on King County grading system - higher == better
* __sqft_above__ square footage of house apart from basement
* __sqft_basement__ square footage of the basement
* __yr_built__ Built Year
* __yr_renovated__ Year when house was renovated
* __zipcode__ zip
* __lat__ Latitude coordinate
* __long__ Longitude coordinate
* __sqft_living15__ Living room area in 2015(implies-- some renovations) This might or might not have affected the lotsize area
* __sqft_lot15__ lotSize area in 2015(implies-- some renovations)

## Explore the data for quality: 

* Check  for nulls
* Check for outliers on data that is numerical

### Check for nulls

* Hint - there is insnull() function as part of dataframe, that you can use to check each field
* Thne you can se sum() across columns (axis = 0) to see how often it happens in each column


In [ ]:
# Check for null values

__What you should see__:

* All the columns have count of nulls at 0

### Check for anomalies on numerical data
* create a list of numerical columns (i.e.int64 and float 64)
    * df.select_dtypes(include=['int64','float64']).columns
    
* run a loop on that list and do a histogram for each (or other visualiation you think would be apt)

__One obvious issue:__
    * Year renovated - appears to use two different standards 

__A few suspicious items:__
    * Price - long tail, which can drive huge absolute errors
    * bedrooms - long-tail and apparently some have 0?
    * bathrooms - some have 0?
    * sqft_living - long-tail
    * sqft_lot - long-tail

__Date is only non-numberical - we should check distribution__

### Data checks

* renovation year
* price
* bedrooms
* bathroom
* sqft_living
* sqft_lot

Things to do:
* Draw histograms but force more granularity (e.g. num of bins)
* do value_counts to see what distribution looks like for outliers
* etc.


__What you should see__:
* Renovation year - looks like it's all OK - 0 means no renovation
* Price - looks like it may be just a few outliers over $3M - need to check how many
* Bedrooms - just 13 with 0 (likely studios) and 1 with 33 - likely outlier
* Bathrooms - 10 with 0 - seems not possible
* sqft_living and sqft_lot - may be just very few outliers - need to check how many

### Checking data - deeper click

* long-tail of prices
* bedrooms - over 30?, 0?
* bathrooms - 0?
* sqftliving - over 8000?
* sqft_lot - over 250K, 500K

* Do counts of issues
* Display problematic ranges (i.e. literally use display)
* Etc - really anything you think will give you a better sens of rht edata

__What you should see:__
    
* Price - looks OK - 45 houses over 3M USD and 8 over 5M USD
* Bedrooms:
    * House with 33 bedrooms is clearly a typo since modest sized home with average price - probably meant as 3
    * Houses with 0 bedrooms - some are suspicious, especially those with 0 bathrooms or those that are huge and multiple bathrooms
* Bathrooms - think 0 bathroom houses make no sense - these should be dropped
* Sqft_living - all makes sense - all outliers have lots of rooms and high price
* Sqft_lot - seems too many with big lots (100+) and those with hugest lots (over 1M) are not ver expensive and very little correlation between lot size and price, even when controlling for size of house. This will likely cause large errors, but let's leave it be

### Checking date distributrion
* Convert all dates to datetime64 type
* Group by year (df_date.dt.year) and month (df_date.dt.year) and count
* Plot as bar

* Or Google until you find the asnwer :)

__What you should see:__

* Looks good, data is ineeded from interval stated and distribution is reasoanble (peak in late spring/early summer in the Northern hemisphere)
* Suspect we're missign some days in May 2015
* It appears there is some seasonality to prices (i.e. lower in winter, higher in summer) - we should add this as a feature in later sprints

## Data clean-up

__To Do__:
* Modify listing with 33 bedrooms to be 3
* Remove listings with 0 bathrooms (simply select sub-section of df where bathrooms > 0)

__WARNING: YOU SHOULD NEVER DO THIS IN PRACTICE. YOU WANT TO FIRST CHECK WITH CLIENT__

## Check data for Linear Regression Suitability
* Check for colinearity
* Check for linear realtionship

### Check for co-linearity

* Create a list of numerical features (Can borrow from our initial data quality check)
* Drop id and price from list
* Use sns.heatmap() and .corr() function of our data-frame with just numberical features to draw correlation matrix



__What you should see__:

* Variables that are highly positively correlated:
    * sqft_above and sqft_living - we should keep sqft_living only
    * sqft_living15 and sqft_lot15 with non-15 values - let's keep non-15 values

* Variables that have high correlation, but we should probably keep:
    * Bathrooms with sqft_living
    * Grade with sqft_living
  
    

### Check for linear relationships

* We should only show relationship for variables we know will have linear relationship and where no correlation:

* Createa a list of numerical variables (Can borrown from previous step)
* remove varialbes that we know can't work or have correlation:
    * Won't work: Lat, long
    * Might work, but requires one-hot encoding: zip-code
    * Corelation: sqft_living15, sqft_lot15
    
* Run a loop and do a sns.jointplot() on price + each variable
    * extra - you could do 1 LR OLS regression plot to see if LR will pick-up on it sns.regplot()

__What you should see and what to do about it__:
* Bedrooms - somewhat liner to 6, but then it's not -> make it one-hot
* Bathrooms - mostly linear -> keep as is
* sqft_living - mostly linear -> keep as is, but add a transofrm to bring large values into line (e.g.)
* sqft_lot - not at all linear for small lots, somewhat linear for big ones -> make it one-hot into several increments
* floors - no relationshop - > drop
* Waterfront - small relationship -> keep
* View - weak relationship ->drop
* Condiiton - no relationship -> drop
* Grade - solid link but not super-linear -> make into one-hot as will be hard to make a transformaiton
* sqft_basement -> the no basement is tripping us up -> requires thoughful FE, let's adress in future sprint
* yr_built - no relationship -> drop
* yr_renovated - would need to look at what it looks like within only those renovated and then decide

__We need to explore just a bit more to figure out what to do with following:__

* sqft-living - how to best transform larger values?
    * Let's try squaring the sqft_living and then normalizing
* sqft_lot - where to draw buckets?
    * let's draw 3 seperate scatters in 50K, 50K to 500K and over 500K
* yr_renovated - is there relationship for those renovated
    * Let's show what non-zero renovation looks like


__What you should see and do:__

* It looks likes squaring the sqft_living makes it far more linear - we should keep
* There is not realtionship in sqft_lot -> we should drop for now
* Yr_renovated -> appears to be a relationship, may make sense to add as a feature in future sprints as one hot (for no renovaiton, and then buckets for decades since renovated)

## Mechanical feature engineering
* We should start with a few features and then add them slowly - use conclusions from above as starting point
* Will need to make some into one-hot (e.g. bedrooms, grade)
* For others will need custom transfrom (E.g. squaring sqft_living)
* For some continous variables, probably a good idea to normalize (e.g. number of bathrooms)
* Some require nothing (e.g. WAterfront - already one-hot :))
* Some are interestign but we will puruse in future sprints (e.g. sqft_basement, yr_renovated, month)

* Since few variables - you can just list each individually and what transformation(s) you want to do


## Train the Linear Model


### Split data

* Let's split at 80/20 train/test

In [ ]:
# The function train_test_split, splits the arguments into two sets.
X_train, X_test, y_train, y_test = None

### Train the model and see regression results on train test-sets

Use statsmodels OLS model

In [ ]:
sm_lm_model = None

# Now, train the model
sm_lm_model_fit_s1 = sm_lm_model.fit()


display(sm_lm_model_fit_s1.summary())




__What you should see__:

* Adjusted R2 of .633 is not bad for first attempt

__Interpreting coefficients__:

* const is 614K which is effectively average price, so coefficients will be things to move it higher or lower

* Coefficients are a little difficult to interpret for some because we transfomed them (E.g. sqft_living is squared + normalized and bathrooms are normalized)

* Most make sense, but a few suspicious items:
    * More bathrooms drives house price down - opposite of what we saw -> this is likely due to co-linearity of sqft_living (0.76 correlation)
    * Grades don't follow a linear pattern at lower end of grade scale (i.e. lowest grade doesn't take away as much value as higher ones oneS) -> likely overfitting
    
* Statistically speaking some should be killed b/c P>[t] is not less then .05 (95% CI)
    * Bedrooms 0 - massive range of where coefficient could be
    * Bedrooms 4,7,8,9,10 and 11 - likely driven by low N
    * Grade 3 and 9 - likely driven by low N
    
* Interesting tid-bits
    * Being waterfront adds 800K dollars to the house value


### Examine accuracy on test data-set
* Run prediciton on un-seen data and see average error on seen and unseen data
    * model_fit.predict()
        * Remember to use sm.addconstant(X_train) so that matrices match
        
* For MAE, you can use sklearn.metrics.mean_absolute_error(y_train, y_predicted)
    * just make sure both matrices are in same shape
* For MAPE, you need to write your own function (Remember formula is Average of ABS(actual-predicted)/true))

In [ ]:
# See how well we did
def mean_absolute_percentage_error(y_true, y_pred): 
    pass

y_lr_train = None
y_lr_test = None

# display(Markdown('###### Performance on training set'))
display('Linear model train set MAE: ${:.0f}'.format(sklearn.metrics.mean_absolute_error(y_train.values.ravel(), y_lr_train.values)))
display('Linear model train set MAPE: {:.2f}%'.format(mean_absolute_percentage_error(y_train.values.ravel(), y_lr_train.values)*100))

# display(Markdown('###### Performance on test set'))
display('Linear model test set MAE: ${:.0f}'.format(sklearn.metrics.mean_absolute_error(y_test.values.ravel(), y_lr_test.values)))
display('Linear model train set MAPE: {:.2f}%'.format(mean_absolute_percentage_error(y_test.values.ravel(), y_lr_test.values)*100))




__What you should see:__

* MAE of ~150,000 dollars
* MAPE of 32% is decent first attempt, but not great
* test set error is close enough to train that not too worried about over-fittign just yet

### Test that model

* Check residuals are normally distributed across all estiamtes
* Check residuals are normally distributed by each point
* Check no heteroskedacity 

* you'll likely need to calcualte residuals in absolute and as percentage in a seperate data-frame
* Then you cna just use histogram and scatter plots to see what's going on

In [ ]:
# Example Provided, but pay attention--you'll have to do this on your own later in the exercise!
df_residuals = y_train.copy()
df_residuals['residual_norm'] = df_residuals.price - y_lr_train.values
df_residuals['residual_perc'] = (df_residuals.price - y_lr_train.values)/df_residuals.price
df_residuals['residual_perc_abs'] = np.abs(df_residuals.price - y_lr_train.values)/df_residuals.price


display("Distribution of residuals")
plt.hist(df_residuals['residual_norm'])
plt.show()
plt.hist(df_residuals['residual_perc'])
plt.show()

display("Residuals vs price")
plt.scatter(df_residuals.price, df_residuals.residual_perc)
plt.show()
display("Residuals vs price for homes under 1M USD")
plt.scatter(df_residuals[df_residuals.price <1000000].price, df_residuals[df_residuals.price <1000000].residual_perc)
plt.show()
display("Residuals vs price for homes over 1M USD")
plt.scatter(df_residuals[df_residuals.price >1000000].price, df_residuals[df_residuals.price >1000000].residual_perc)
plt.show()




__What you should see:__

* We definitely have a few problems here
* Residuals are not normally distribvuted - more negative errors
* Residuals are not normally distributed across points - very negative skew at lower prices
* There is definite heteroskedacity - bigger errors (%wise) for smaller price points - i.e. we think house should be priced up 3x more then actual sale price

## Explore error

* Let's see if these high-error low-priced homes have something in common
* Basically add residual in percentage to the df_transformed_features_s1
* Then predict all prices
* Then calculate residuals
* Then for each feature do a histogram of distribution in two steps:
    * Do it only on homes under 1,000,000 USD 
    * Plot seperte distributions for high-error and low-error homes (let's dre the bad error at more then 75%)
   

In [ ]:
# Example provided, but pay attention! You'll be doing this yourself later!

df_error = df_transformed_features_s1.copy()
df_error['price'] = numpy_transform_target_s1
df_error["price_predicted"] = sm_lm_model_fit_s1.predict(sm.add_constant(df_transformed_features_s1))
df_error['residual_perc'] = (df_error.price - df_error.price_predicted)/df_error.price

df_error = df_error[df_error.price < 1000000]

display(df_error.head(10))

features_list = list(df_error.columns)
features_list.remove('price')
features_list.remove('price_predicted')
features_list.remove('residual_perc')

for column in features_list:
    plt.hist(df_error[df_error['residual_perc'] > -0.75][column], alpha=0.5, label='Normal Error',density = True)
    plt.hist(df_error[df_error['residual_perc'] < -0.75][column], alpha=0.5, label='Big Error',density = True)
    plt.legend(loc='upper right')
    plt.title(column)
    plt.show()
        

__What you should see:__

* Nothing really jumps out - distribution of things with normal errors and really bad error is pretty similar
* It's probably that one of the missing variables is playing a big role
* We should investigate that in sprint 2 or 3


# Sprint 2

* Add in remaining features from sprint 1
* Re-run OLS LR
* Remove features that are not statistically significant
* Re-run OLS LR
* Join new data and explore if predicitive
* Add new data to the data-set
* Re-run OLS LR
* Re-run with XGBoost

## Add in remaining features from sprint 1
* yr_renovated
    * create fucntion to bucket in an intelligent way (by decade)
    * create transform functionto apply along an entire series

* sqft_basement
    * create fucntion to bucket in an intelligent way
    * create transform functionto apply along an entire series
    
* month one-hot
    * convert date column to datetiem64
    * create new column month that is just month porton of date

* add new variables to dataframe mapper
* run dataframe mapper
    


In [ ]:
# Feature engineering!!!


## Re-run OLS Linear Regression

### Train OLS LR with new features

In [ ]:
# The function train_test_split, splits the arguments into two sets.
X_train, X_test, y_train, y_test = None

sm_lm_model_s2_1 = None

sm_lm_model_fit_s2_1 = None

display(sm_lm_model_fit_s2_1.summary())


__What you should see:__
* Adjsut R2 got better (.65 from .63)
* We have lots of coefficients that are not statistically significant

* Let's run MAE and MAPE to see if it improves accuracy


### Test re-run OLS regression

* Predict train and test
* Calculate MAPE and MAE

In [ ]:
y_lr_train = None
y_lr_test = None

# display(Markdown('###### Performance on training set'))
display('Linear model train set MAE: ${:.0f}'.format(sklearn.metrics.mean_absolute_error(y_train.values.ravel(), y_lr_train.values)))
display('Linear model train set MAPE: {:.4f}%'.format(mean_absolute_percentage_error(y_train.values.ravel(), y_lr_train.values)))

# display(Markdown('###### Performance on test set'))
display('Linear model test set MAE: ${:.0f}'.format(sklearn.metrics.mean_absolute_error(y_test.values.ravel(), y_lr_test.values)))
display('Linear model train set MAPE: {:.4f}%'.format(mean_absolute_percentage_error(y_test.values.ravel(), y_lr_test.values)))

__What you should see:__

* Our MAPE got a bit better - from 31.6% to 30.7% - YAY

### Remove variables not statistically significant 

* You can get pvalues from paramter pvalues
* Create a list of pvalues that are > 0.05 (this will be a PD Series with feature name + pvalues)
* Get just names of those features
* using pandas drop(feature_name, axis = 1, inplace = True) - drop those features from our df_transformed_features

In [ ]:
# Code provided for you, so you can see how this is done
drop_list = sm_lm_model_fit_s2_1.pvalues[sm_lm_model_fit_s2_1.pvalues > 0.05]
display(drop_list)
drop_list_names = drop_list.index

display(len(drop_list_names))

df_transformed_features_s2_1 = df_transformed_features_s2_1.copy()
display(df_transformed_features_s2_1.shape)

for fe_to_drop in drop_list_names:
        df_transformed_features_s2_1.drop(fe_to_drop, axis = 1, inplace = True)

display(df_transformed_features_s2_1.shape)

display(df_transformed_features_s2_1.head(10))

### Re-run OLS with statistically significant variables only

In [ ]:
# The function train_test_split, splits the arguments into two sets.
X_train, X_test, y_train, y_test = None

sm_lm_model_s2_1 = sm.OLS(y_train, sm.add_constant(X_train))

# Fit the model
sm_lm_model_fit_s2_1 = None

# Get the summary
display(sm_lm_model_fit_s2_1.summary())


__What yuou should see:__

* Adjusted R square went down a tiny bit (as expected)
* Some new features became statistically insignificant 

### Test MAPE, MAE of OLS Regression

In [ ]:
# Get predictions for training and testing sets
y_lr_train = None
y_lr_test = None

# display(Markdown('###### Performance on training set'))
display('Linear model train set MAE: ${:.0f}'.format(sklearn.metrics.mean_absolute_error(y_train.values.ravel(), y_lr_train.values)))
display('Linear model train set MAPE: {:.4f}%'.format(mean_absolute_percentage_error(y_train.values.ravel(), y_lr_train.values)))

# display(Markdown('###### Performance on test set'))
display('Linear model test set MAE: ${:.0f}'.format(sklearn.metrics.mean_absolute_error(y_test.values.ravel(), y_lr_test.values)))
display('Linear model train set MAPE: {:.4f}%'.format(mean_absolute_percentage_error(y_test.values.ravel(), y_lr_test.values)))

__What you should see:__

* Accuracy went down a little bit, but not as bad as our first regression

### Explore error

Let's see if adding new data helped with heteroskedacity

__What you should see:__

* It didn't really help much...

## Add new data (income-level socioeconomic data)
* Load blockgroup-data
    * Blockgroup_avg_income_kc.csv
* Load csv that has transaltion function between lat-long and blockgroup
    * kc_blockgroup_lat_long.csv
* Join data:
    * Convert Lat-long to blockgroup
    * Join relevant blockgroup_demographic
* Explore data
* Re-run with LR

### Load blockgroup-data & lat_long pairs translation function





In [ ]:
df_blockgroup_data = None

display(df_blockgroup_data.shape)
display(df_blockgroup_data.head(10))

df_blockgroup_pairs = None
display(df_blockgroup_pairs.head(10))


### Join lat_long to blockgroup data to enable exploration

* Use merge to join blocgkroup_date with income and lat-long blockgroup dataframe
* Test for nulls and such

In [ ]:
df_blockgorup_joined = None

### Explore data
* Compare lat-long income with lat-long average price of home per squarefoot
* Fun hack - you can just use scatter since lat and long are nicely linear (use long for X, and lat for y)
    * Then use color to see median income or average price
    * For housing data - probably best to color not by price but dollars per square foot of living space

__What you should see:__

* there is a lot more disperity between two charts, but same general gradient
* Red is high, blue is low and we see similar hot-spots (i.e. around downtown Seattle)

## Re-run OLS with new Data

### Add lat_long_income into our data_set 

* Merge using multiple varialbes - you can simply supply left_on and right_on as a list of features to match
* check for nulls or that you got too many when you merge

__WARNING: You'll have 13 nulls, just drop for right now, but right thing to do would be to take average of 3 neareast ones...but we don't have time for that__

### Run new data set through our pipeline

In [ ]:
# Use DataFrameMapper!
preprocessing_steps_v4 = None


mapper_features_v4 = None


np_transformed_features_s2_2 = mapper_features_v4.fit_transform(df_clean_merge.copy())
df_transformed_features_s2_2 = pd.DataFrame(data = np_transformed_features_s2_2, columns = mapper_features_v4.transformed_names_)


mapper_target_s2_2 = DataFrameMapper([(['price'],None)])
numpy_transform_target_s2_2 = mapper_target_s2_2.fit_transform(df_clean_merge[[TARGET]].copy())

df_transformed_target_s2_2 = pd.DataFrame(data = numpy_transform_target_s2_2, columns = ['price'])

### Train using new data with OLS, show coefficients and also calculate MAPE and MAE

In [ ]:
# The function train_test_split, splits the arguments into two sets.
X_train, X_test, y_train, y_test = None

# X_train.isnull().sum(axis = 0)

# Train an OLS model from sm on the training data
sm_lm_model_s2_2 = None

# Now, fit the model (remember, statsmodel API is different than sklearn's!)
sm_lm_model_fit_s2_2 = None


# Now, display the summary from the model
display(sm_lm_model_fit_s2_2.summary())

# Get predictions for training and testing sets
y_lr_train = None
y_lr_test = None

# display(Markdown('###### Performance on training set'))
display('Linear model train set MAE: ${:.0f}'.format(sklearn.metrics.mean_absolute_error(y_train.values.ravel(), y_lr_train.values)))
display('Linear model train set MAPE: {:.4f}%'.format(mean_absolute_percentage_error(y_train.values.ravel(), y_lr_train.values)*100))

# display(Markdown('###### Performance on test set'))
display('Linear model test set MAE: ${:.0f}'.format(sklearn.metrics.mean_absolute_error(y_test.values.ravel(), y_lr_test.values)))
display('Linear model train set MAPE: {:.4f}%'.format(mean_absolute_percentage_error(y_test.values.ravel(), y_lr_test.values)*100))

__What you should see:__
    
* Adjusted R2 improved by ~5% which is also very good
* MAPE improved by ~4% which is pretty good

### Explore error

In [ ]:
# See if you can figure out how to explore your residuals. If you get stuck, take a look at the solution branch!






__What you should see:__

* We continue to struggle with homoskedcity

## Try more sophisticated model - XGBoost

### Re-train XGRegressor using existing data pipeline 

In [ ]:
# from xgboost import XGBRegressor
XG_model = None

# Fit your model
XG_model.fit(X_train, y_train)


# Generate training set predictions
y_xg_train = None
y_xg_train = np.reshape(y_xg_train, (-1, 1))

# Generate test set predictions
y_xg_test = None
y_xg_test = np.reshape(y_xg_test, (-1, 1))


display(Markdown('###### Performance on training set'))

display('XGBoost model train set MAE: ${:.0f}'.format(sklearn.metrics.mean_absolute_error(y_train, y_xg_train)))
display('XGBoost model train set MAPE: {:.2f}%'.format(mean_absolute_percentage_error(y_train.values, y_xg_train)*100))

display(Markdown('###### Performance on test set'))

display('XGBoost model test set MAE: ${:.0f}'.format(sklearn.metrics.mean_absolute_error(y_test, y_xg_test)))
display('XGBoost model train set MAPE: {:.2f}%'.format(mean_absolute_percentage_error(y_test.values, y_xg_test)*100))

display(Markdown('###### Coefficients'))

FEATURES = mapper_features_v4.transformed_names_

pd.DataFrame({'features': FEATURES, 'importance': XG_model.feature_importances_}).sort_values('importance', ascending=False)

__What you should see:__

* MAPE of about 26% - only 0.5% better than the linear regression
* Coefficient stength - looks like average income raises to the top, so it's possible that XGB could find some value in some of the data we decided to leave out - we should try that next


### Add a bit more data into our pipeline

* lat, long
* view - was weak relationship



In [ ]:
# Use DataFrameMapper!
preprocessing_steps_s2_3 = None


mapper_features_s2_3 = None

np_transformed_features_s2_3 = mapper_features_s2_3.fit_transform(df_clean_merge.copy())
df_transformed_features_s2_3 = pd.DataFrame(data = np_transformed_features_s2_3, columns = mapper_features_s2_3.transformed_names_)

### Re-train XGBoost, show coefficients and calculate MAPE and MAE

In [ ]:
# The function train_test_split, splits the arguments into two sets.
X_train, X_test, y_train, y_test = None

# Fit the model

# Get predictions on your training set
y_xg_train = None
y_xg_train = np.reshape(y_xg_train, (-1, 1))

# Get predictions on your testing set
y_xg_test = None
y_xg_test = np.reshape(y_xg_test, (-1, 1))


display(Markdown('###### Performance on training set'))

display('XGBoost model train set MAE: ${:.0f}'.format(sklearn.metrics.mean_absolute_error(y_train, y_xg_train)))
display('XGBoost model train set MAPE: {:.2f}%'.format(mean_absolute_percentage_error(y_train.values, y_xg_train)*100))

display(Markdown('###### Performance on test set'))

display('XGBoost model test set MAE: ${:.0f}'.format(sklearn.metrics.mean_absolute_error(y_test, y_xg_test)))
display('XGBoost model train set MAPE: {:.2f}%'.format(mean_absolute_percentage_error(y_test.values, y_xg_test)*100))

display(Markdown('###### Coefficients'))

FEATURES = mapper_features_s2_3.transformed_names_

pd.DataFrame({'features': FEATURES, 'importance': XG_model.feature_importances_}).sort_values('importance', ascending=False)


__What you should see:__

* Huge improvement - added 10%+ of accuracy and lat, and long are 2nd and 3rd most importnat feature
* view climbed pretty high to the list of things that matter

### Explore error

In [ ]:
# See if you can figure out how to explore your residuals. If you get stuck, take a look at the solution branch!



__What you should see:__

* Remarkably our issue with homes that are less expensive exists even with xgboost
* It must be that we're simply missing an important predictor variable 

## Hyperparameter tuning

### Let's play with depth only

* Create a list of depths (e.g. from 3 to 15)
* Run for-loop wher we re-train with different depth
* See which one does best

In [ ]:
# Example provided, you'll tune the next hyperparameter yourself in the cells below!

max_depth_list = list(range(3,15))
test_MAPE = []

for max_depth in max_depth_list:
    XG_model_tuned = XGBRegressor(max_depth = max_depth)
    XG_model_tuned.fit(X_train, y_train)
    
    y_xg_test = XG_model_tuned.predict(X_test)
    y_xg_test = np.reshape(y_xg_test, (-1, 1))
    
    MAPE = mean_absolute_percentage_error(y_test.values, y_xg_test)*100
    display('Trained with max-depth of {:0f} and MAPE is {:2f}%'.format(max_depth,MAPE ))
    test_MAPE.append(MAPE)
    
plt.plot(max_depth_list, test_MAPE)
    


#     display(Markdown('### Max_Depth = {:0f}'.format(max_depth_increment)))
#     display(Markdown('###### Performance on test set'))

#     display('XGBoost model test set MAE: ${:.0f}'.format(sklearn.metrics.mean_absolute_error(y_test, y_xg_test)))
#     display('XGBoost model test set MAPE: {:.2f}%'.format(mean_absolute_percentage_error(y_test.values, y_xg_test)*100))

__What you should see:__

* loks like depth of 10 is pretty sweet - reduces error by ~3%
* this is not surprising at all since suspect lat and long play occupy a large part of those trees to isolate neighbourhoods where similar houses get higher price

### Let's try to play with n_estimators

* similar to above, go from 50 to 500 in incremenets of 50

In [ ]:
estimators_list = None
test_MAPE = []


# Now, loop through all the numbers in estimators_list. 
for None in None:
    # For each iteration, use that number as the n_estimators parameter. Leave max_depth as 10
    XG_model_tuned = None
    # Fit the model
    
    
    
    # Generate predictions
    y_xg_test = None
    
    # This line reshapes the vector of predictions that the last line returned. 
    y_xg_test = np.reshape(y_xg_test, (-1, 1))
    
    # Calculate MAPE 
    MAPE = None
    display('Trained with n_estimators of {:0f} and MAPE is {:2f}%'.format(n_estimators,MAPE ))
    
    # append the MAPE value to test_MAPE, so that we can track the MAPE performance of each iteration 
    
    
plt.plot(estimators_list, test_MAPE)
    


#     display(Markdown('### Max_Depth = {:0f}'.format(max_depth_increment)))
#     display(Markdown('###### Performance on test set'))

#     display('XGBoost model test set MAE: ${:.0f}'.format(sklearn.metrics.mean_absolute_error(y_test, y_xg_test)))
#     display('XGBoost model test set MAPE: {:.2f}%'.format(mean_absolute_percentage_error(y_test.values, y_xg_test)*100))

__What you should see:__

* n_estimators of 250 seems like the sweet spot, but only reduced error by 0.07%
* Our new error is now 13%
* We should stop with hyper-parameter tuning now...